# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)
# join the file path and roots with the subdirectories using glob

/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data
['/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-08-events.csv', '/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-27-events.csv', '/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-11-events.csv', '/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-25-events.csv', '/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-21-events.csv', '/home/gulbulut/repos/udacity-data-engineering-nanodegree-projects/Project_2_Data_Modeling_with_Cassandra/event_data/2018-11-15-events.csv', '/home/gulbulut/repos/udacity-data-en

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e) 

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Solution 1
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(sessionId text, itemInSession text ,artist text,  length text,song_title text,  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library ( sessionId, itemInSession,artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[8],line[3],line[0], line[9],line[5]))

#### Query Description: sessionId and itemInSession columns together as primary key, however partition key is sessionId.

####  Print out the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
query = "select artist,length,song_title from music_library WHERE sessionId = '338' AND itemInSession = '4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.length,row.song_title)

Faithless 495.3073 Music Matters (Mark Knight Dub)


In [11]:
## Solution 2

query = "CREATE TABLE IF NOT EXISTS user_library "
query = query + "(userid text,sessionId text,itemInSession text , artist text,  first_name text,last_name text,  song text,  PRIMARY KEY ((userid,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)



                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (userid,sessionId,itemInSession,artist, first_name,last_name,song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10],line[8],line[3],line[0], line[1],line[4],line[9],))
                    

#### Query Description:  userId and SessionId columns are together as primary key, however partition key is userId.
#### The clustering column is itemInSession. The data is sorted in ascending order since itemInSession is clustering column.
#### Print out only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
query = "select artist,song, first_name, last_name from user_library WHERE userid = '10' AND sessionId = '182'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.song,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## Solution 3
query = "CREATE TABLE IF NOT EXISTS one_song_library "
query = query + "(  song text, userid text,first_name text,last_name text, PRIMARY KEY (song, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO one_song_library (song,userid,first_name,last_name,)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],line[10],line[1],line[4]))

#### Query Description: Song and userid columns together as primary key, however partition key is song.

#### Print out every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query = "select first_name, last_name from one_song_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table one_song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()